In [1]:
import numpy as np
#import pickle as pkl
import pandas as pd
from players import Player
import mechs
import inequality
np.set_printoptions(precision=2)

In [2]:
# open a file, where you stored the pickled data
classes = pd.read_excel('CMData/2015C/2015C.xlsx', names=['id'], usecols='A')
#print(classes)

In [3]:
students = pd.read_json('CMData/2015C/instance10.in', lines=True, nrows = 1)['students'][0]

In [20]:
util_matrix = {}
n_students = 50
i = 0
normalize = True

for student in students[:n_students]:
    for idx, preference in enumerate(student['utils']):
        if len(preference) <= 2:
            pass
        else:
            print('Deleting: ' + str(preference))
            del student['utils'][idx]
    if normalize:
        try:
            normed_utils = np.maximum(student['utils'], np.zeros((len(student['utils']), 2)))
            denom = np.sum(normed_utils[:,1])
            normed_utils[:,1] *= (1. / denom)        
            util_matrix[i] = {}
            for pair in normed_utils:
                util_matrix[i][pair[0]] = pair[1]
            i = i + 1

        except:
            n_students -= 1
    else:
        util_matrix[i] = {}
        for pair in student['utils']:
            util_matrix[i][pair[0]] = max(pair[1], 0)
        i = i + 1
    
df = pd.DataFrame(util_matrix)
n_classes = len(df)
print(n_students)

Deleting: [1439, 1572, 100]
Deleting: [1478, 1570, -20]
Deleting: [1535, 1542, 200]
Deleting: [1454, 1516, -2000]
48
dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47])


/tmp/ipykernel_12402/1837074343.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  normed_utils = np.maximum(student['utils'], np.zeros((len(student['utils']), 2)))


In [21]:
player_dict = {}
for (player_id, utils) in df.iteritems():
    player_dict[player_id] = Player(player_id, u_i = utils.values, c_i = 0.1, g = n_classes, n = n_students, U = df.to_numpy().T)

alloc = mechs.usw(player_dict, n_classes)
alloc_v = mechs.usw_v(player_dict, n_classes)

48


In [22]:
fs_u = inequality.FehrSchmidtIneq(player_dict, alloc)
fs_v = inequality.FehrSchmidtIneq(player_dict, alloc_v)
print(fs_u)
print(fs_v)

12.971287097851594
11.18488104799535
